This notebook is the third part of [RANZCR 1st Place Solution by TF](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-1-make-masks).
This notebook is based on [RANZCR 1st Place Soluiton Seg Model (small ver.)](https://www.kaggle.com/haqishen/ranzcr-1st-place-soluiton-seg-model-small-ver).

The third step is to generate masks by using the trained segmentation model in the [previous step](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-2-seg-model). Then, TFRecord files are made from images, generated masks, labels, and folds. The files will be used to train the classification model at the [next step](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-4-cls-model).

The original notebook uses 5 fold results to make masks. In this notebook, only 1 fold is used, because it took long time (about 4.5 hours) to train the segmentation model.
I used the result of [Version 10](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-2-seg-model?scriptVersionId=61479593).

## Install Segmentation Models Locally

In [ ]:
%env SM_FRAMEWORK=tf.keras
!pip install ../input/segmentation-models-keras/Keras_Applications-1.0.8-py3-none-any.whl --quiet
!pip install ../input/segmentation-models-keras/image_classifiers-1.0.0-py3-none-any.whl --quiet
!pip install ../input/segmentation-models-keras/efficientnet-1.0.0-py3-none-any.whl --quiet
!pip install ../input/segmentation-models-keras/segmentation_models-1.0.1-py3-none-any.whl --quiet

print("Segmentation Models installed.")

## Config and Libraries

In [ ]:
DEBUG = False

In [ ]:
# libraries
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import segmentation_models as sm
from kaggle_datasets import KaggleDatasets

print(tf.__version__)

In [ ]:
train_image_size = 1024
gen_image_size = 512
batch_size = 16
jpeg_quality = 100

In [ ]:
sm.set_framework('tf.keras')
tf.keras.backend.set_image_data_format('channels_last')

## TPU

In [ ]:
try: # detect TPUs
    # NEW: in Tensorflow 2.4
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() 
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # otherwise detect GPUs
    strategy = tf.distribute.MirroredStrategy() # single-GPU or multi-GPU
    
print(f"Running on {strategy.num_replicas_in_sync} replicas")

In [ ]:
ranzcr_name = 'ranzcr-clip-catheter-line-classification'
ranzcr_fold_dir = '../input/ranzcr-fold/'
ranzcr_model_dir = '../input/ranzcr-1st-place-solution-by-tf-models/'

In [ ]:
seg_model_name = 'seg_model_V10_0.hdf5'

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path(ranzcr_name)

GCS_DS_PATH

## Train_V2

In [ ]:
df_train = pd.read_csv(ranzcr_fold_dir + 'train_v2.csv')

df_train

In [ ]:
uid_fold_dict = dict(
    zip(df_train['StudyInstanceUID'], df_train['fold']))

len(uid_fold_dict)

## Train TFRecords

In [ ]:
def decode_train_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    return image

def read_train_tfrecord(example):
    TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'StudyInstanceUID': tf.io.FixedLenFeature([], tf.string),
        'ETT - Abnormal': tf.io.FixedLenFeature([], tf.int64),
        'ETT - Borderline': tf.io.FixedLenFeature([], tf.int64),
        'ETT - Normal': tf.io.FixedLenFeature([], tf.int64),
        'NGT - Abnormal': tf.io.FixedLenFeature([], tf.int64),
        'NGT - Borderline': tf.io.FixedLenFeature([], tf.int64),
        'NGT - Incompletely Imaged': tf.io.FixedLenFeature([], tf.int64),
        'NGT - Normal': tf.io.FixedLenFeature([], tf.int64),
        'CVC - Abnormal': tf.io.FixedLenFeature([], tf.int64),
        'CVC - Borderline': tf.io.FixedLenFeature([], tf.int64),
        'CVC - Normal': tf.io.FixedLenFeature([], tf.int64),
        'Swan Ganz Catheter Present': tf.io.FixedLenFeature([], tf.int64),
    }
    
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_train_image(example['image'])
    study_inst_id = example['StudyInstanceUID']
    ett_abnormal = example['ETT - Abnormal']
    ett_borderline = example['ETT - Borderline']
    ett_normal = example['ETT - Normal']
    ngt_abnormal = example['NGT - Abnormal']
    ngt_borderline = example['NGT - Borderline']
    ngt_inc_imaged = example['NGT - Incompletely Imaged']
    ngt_normal = example['NGT - Normal']
    cvc_abnormal = example['CVC - Abnormal']
    cvc_borderline = example['CVC - Borderline']
    cvc_normal = example['CVC - Normal']
    swan_ganz_cat_present = example['Swan Ganz Catheter Present']
    labels = [
        ett_abnormal, ett_borderline, ett_normal,
        ngt_abnormal, ngt_borderline, ngt_inc_imaged, ngt_normal,
        cvc_abnormal, cvc_borderline, cvc_normal,
        swan_ganz_cat_present,
    ]
    return image, labels, study_inst_id

def load_train_dataset(filenames):
    ds = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE)
    ds = ds.map(read_train_tfrecord, num_parallel_calls=AUTOTUNE)
    return ds

## Model

In [ ]:
def load_model(model_name):
    with strategy.scope():
        model_path = ranzcr_model_dir + model_name
        model = tf.keras.models.load_model(model_path)

    model.summary()
    return model

In [ ]:
model = load_model(seg_model_name)

## Dataset

In [ ]:
def make_predict_dataset(image_batch):
    def _preprocess_image(image):
        image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
        image = tf.image.resize(image, (train_image_size, train_image_size))
        return image
    
    ds = tf.data.Dataset.from_tensor_slices(image_batch)
    ds = ds.map(_preprocess_image, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    return ds

In [ ]:
def convert_image(image):
    image = tf.image.resize(image, (gen_image_size, gen_image_size))
    image = tf.cast(image, dtype=tf.uint8)
    image = tf.image.encode_jpeg(image, quality=jpeg_quality)
    return image

def convert_mask(mask):
    mask = tf.image.resize(mask, (gen_image_size, gen_image_size))
    # Generated mask is 0..1, so change to 0..255.
    mask = mask * 255.0
    # Make 3 channels for encoding as PNG.
    zeros = tf.zeros((gen_image_size, gen_image_size, 1))
    mask = tf.concat([mask, zeros], axis=-1) 
    mask = tf.cast(mask, dtype=tf.uint8)
    mask = tf.io.encode_png(mask)
    return mask

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        # BytesList won't unpack a string from an EagerTensor.
        value = value.numpy() 
    elif isinstance(value, str):
        # string needs to be encoded to bytes.
        value = value.encode('utf-8')
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(image, mask, labels, fold):
    feature = {
        'image': _bytes_feature(image),
        'mask': _bytes_feature(mask),
        'ETT - Abnormal': _int64_feature(labels[0]),
        'ETT - Borderline': _int64_feature(labels[1]),
        'ETT - Normal': _int64_feature(labels[2]),
        'NGT - Abnormal': _int64_feature(labels[3]),
        'NGT - Borderline': _int64_feature(labels[4]),
        'NGT - Incompletely Imaged': _int64_feature(labels[5]),
        'NGT - Normal': _int64_feature(labels[6]),
        'CVC - Abnormal': _int64_feature(labels[7]),
        'CVC - Borderline': _int64_feature(labels[8]),
        'CVC - Normal': _int64_feature(labels[9]),
        'Swan Ganz Catheter Present': _int64_feature(labels[10]),
        'fold': _int64_feature(fold),
    }
    
    example_proto = tf.train.Example(
        features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
train_tfrec_paths = GCS_DS_PATH + '/train_tfrecords/*.tfrec'
train_tfrec_file_names = sorted(tf.io.gfile.glob(train_tfrec_paths))
train_tfrec_file_names = \
    train_tfrec_file_names[:2] if DEBUG else train_tfrec_file_names

len(train_tfrec_file_names)

Only one fold...

In [ ]:
for train_file_i, train_file_name in enumerate(train_tfrec_file_names):
    train_ds = load_train_dataset(train_file_name)
    train_ds = train_ds.take(100) if DEBUG else train_ds
    
    gen_item_count = 0
    for train_item in train_ds:
        gen_item_count += 1
    gen_file_name = \
        "{0:02d}-{1:04d}.tfrec".format(train_file_i, gen_item_count)

    print("Writing {0}...".format(gen_file_name))
    with tf.io.TFRecordWriter(gen_file_name) as writer:
        train_batch_ds = train_ds.batch(batch_size)
        for image_batch, labels_batch, study_inst_id_batch in train_batch_ds:
            print('.', end='', flush=True)
            pred_ds = make_predict_dataset(image_batch)
            mask_batch = model.predict(pred_ds, verbose=0)
            for image, mask, labels, study_inst_id in \
                    zip(image_batch, mask_batch,
                        labels_batch, study_inst_id_batch):
                image = convert_image(image)
                mask = convert_mask(mask)
                uid = study_inst_id.numpy().decode('utf-8')
                fold = uid_fold_dict[uid]
                example = serialize_example(image, mask, labels, fold)
                writer.write(example)
    print()

In [ ]:
! ls -l

## Verify Generated TFRecords

In [ ]:
def decode_gen_image(image_bytes):
    image = tf.image.decode_jpeg(image_bytes, channels=3)
    return image

def decode_gen_mask(mask_bytes):
    mask = tf.io.decode_png(mask_bytes, channels=3)
    return mask

def read_gen_tfrecord(example):
    TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'mask': tf.io.FixedLenFeature([], tf.string),
        'ETT - Abnormal': tf.io.FixedLenFeature([], tf.int64),
        'ETT - Borderline': tf.io.FixedLenFeature([], tf.int64),
        'ETT - Normal': tf.io.FixedLenFeature([], tf.int64),
        'NGT - Abnormal': tf.io.FixedLenFeature([], tf.int64),
        'NGT - Borderline': tf.io.FixedLenFeature([], tf.int64),
        'NGT - Incompletely Imaged': tf.io.FixedLenFeature([], tf.int64),
        'NGT - Normal': tf.io.FixedLenFeature([], tf.int64),
        'CVC - Abnormal': tf.io.FixedLenFeature([], tf.int64),
        'CVC - Borderline': tf.io.FixedLenFeature([], tf.int64),
        'CVC - Normal': tf.io.FixedLenFeature([], tf.int64),
        'Swan Ganz Catheter Present': tf.io.FixedLenFeature([], tf.int64),
        'fold': tf.io.FixedLenFeature([], tf.int64),
    }
    
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_gen_image(example['image'])
    mask = decode_gen_mask(example['mask'])
    ett_abnormal = example['ETT - Abnormal']
    ett_borderline = example['ETT - Borderline']
    ett_normal = example['ETT - Normal']
    ngt_abnormal = example['NGT - Abnormal']
    ngt_borderline = example['NGT - Borderline']
    ngt_inc_imaged = example['NGT - Incompletely Imaged']
    ngt_normal = example['NGT - Normal']
    cvc_abnormal = example['CVC - Abnormal']
    cvc_borderline = example['CVC - Borderline']
    cvc_normal = example['CVC - Normal']
    swan_ganz_cat_present = example['Swan Ganz Catheter Present']
    labels = [
        ett_abnormal, ett_borderline, ett_normal,
        ngt_abnormal, ngt_borderline, ngt_inc_imaged, ngt_normal,
        cvc_abnormal, cvc_borderline, cvc_normal,
        swan_ganz_cat_present,
    ]
    fold = example['fold']
    return image, mask, labels, fold

def load_gen_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=None)
    dataset = dataset.map(read_gen_tfrecord, num_parallel_calls=None)
    return dataset

In [ ]:
gen_tfrec_file_names = tf.io.gfile.glob('*.tfrec')
gen_ds = load_gen_dataset(gen_tfrec_file_names)

print(gen_ds)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 20,10

f, axarr = plt.subplots(1,5)
masks = []
gen_ds_iter = iter(gen_ds)
for p in range(5):
    img, mask, labels, fold = next(gen_ds_iter)
    axarr[p].imshow(img)
    title = "{0}{1}{2}:{3}{4}{5}{6}:{7}{8}{9}:{10}-{11}".format(
        labels[0], labels[1], labels[2],
        labels[3], labels[4], labels[5], labels[6],
        labels[7], labels[8], labels[9], labels[10],
        fold)
    axarr[p].set_title(title)
    masks.append(mask)

f, axarr = plt.subplots(1,5)
for p in range(5):
    axarr[p].imshow(masks[p][ : , : , 0])

f, axarr = plt.subplots(1,5)
for p in range(5):
    axarr[p].imshow(masks[p][ : , : , 1])